### Calculate TF-IDF

In [85]:
%run preprocessing.ipynb

In [86]:
%%time
# obtain term-frequency matrix
import numpy as np
import os
import json
word2index = {}
document2index = {}
index2document = {}
document_word_vectors = {}
w_cnt = 0
d_cnt = 0
for root, dirs, files in os.walk('./data/paragraph/'):
    for f in files:
        document_word_vectors[f] = []
        document2index[f] = d_cnt
        index2document[d_cnt] = f
        d_cnt+=1
        with open(root+'/'+f) as fs:
            try:
                text = fs.read()
                words = textPreprocess(text)
                for w in words:
                    if w not in word2index:
                        word2index[w] = w_cnt
                        w_cnt+=1
                    document_word_vectors[f].append(word2index[w])                
                #for line in fs:
                #    obj = json.loads(line)
                #    textType = obj['type']
                #    if textType == 'paragraph':
                #        words = textPreprocess(obj['content'])
                #        for w in words:
                #            if w not in word2index:
                #                word2index[w] = w_cnt
                #                w_cnt+=1
                #            document_word_vectors[f].append(word2index[w])
            except:
                print (f)
                        
w_f_matrix = np.zeros((len(word2index),len(document2index)))
for doc in document_word_vectors:
    i = document2index[doc]
    for j in document_word_vectors[doc]:
        w_f_matrix[j,i]+=1 

CPU times: user 1.9 s, sys: 16 ms, total: 1.92 s
Wall time: 1.92 s


In [87]:
w_f_matrix.shape

(25377, 9)

In [88]:
# obtain normalized term-frequency matrix
t_f = np.copy(w_f_matrix)
sum_f = np.zeros(len(document2index))
for i in range(len(document2index)):
    sum_f[i] = np.sum(t_f[:,i])
t_f = np.divide(t_f,sum_f)  

In [89]:
# obtaining tf-idf matrix
inv_doc_freq = np.count_nonzero(t_f,axis=1)
def normalize(a,x):
    return np.log(x/a)
norm = np.vectorize(normalize)
inv_doc_freq = norm(inv_doc_freq,len(document2index))
tf_idf = np.multiply(t_f,inv_doc_freq.reshape(-1,1))

In [90]:
tf_idf

array([[2.08042101e-05, 1.01532723e-05, 2.16941632e-05, ...,
        0.00000000e+00, 1.34306036e-05, 3.38327372e-05],
       [1.07426947e-04, 0.00000000e+00, 0.00000000e+00, ...,
        8.21278323e-05, 0.00000000e+00, 7.76455588e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.05190759e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.05190759e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.05190759e-04]])

In [91]:
index = np.argpartition(tf_idf[:,2], -4)[-4:]
index = index[np.argsort(tf_idf[:, 2][index])]
index

array([2657,   49, 2493,   50])

In [92]:
#https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array

def find_top_k_words(tf_idf, k):
    for i in range(tf_idf.shape[1]):
        freq_words = []
        index = np.argpartition(tf_idf[:, i], -k)[-k:]
        index = index[np.argsort(tf_idf[:, i][index])]
        print('Most frequent words in: ', index2document[i])
        for ind in index:
            freq_words.append(list(word2index.keys())[list(word2index.values()).index(ind)])
        print(freq_words)
            
            
    

In [93]:
find_top_k_words(tf_idf, 10)

Most frequent words in:  BVB-AnnualReport-2016.txt
['iduna', '20152016', 'saison', 'bvb', 'fc', 'league', 'uefa', 'teur', 'borussia', 'dortmund']
Most frequent words in:  CarlZeissMeditec-AnnualReport-2017.txt
['ärzte', 'veracity', 'devices', 'ophthalmic', 'patienten', 'tsd', '201617', 'meditec', 'carl', 'zeiss']
Most frequent words in:  BVB-AnnualReport-2015.txt
['mannschaft', 'league', 'iduna', 'uefa', 'saison', 'eur', '20142015', 'borussia', 'teur', 'dortmund']
Most frequent words in:  BVB-AnnualReport-2017.txt
['bvb', 'uefa', 'iduna', 'saison', '20162017', '20172018', 'eur', 'borussia', 'teur', 'dortmund']
Most frequent words in:  BMW-AnnualReport-2015.txt
['vorzugsaktien', 'fahrzeug', 'modelle', 'motorräder', 'fahrzeuge', 'mio', 'mini', 'group', 'automobile', 'bmw']
Most frequent words in:  BMW-AnnualReport-2017.txt
['rollsroyce', 'mio', 'vorzugsaktien', 'motorräder', 'fahrzeug', 'group', 'fahrzeuge', 'mini', 'automobile', 'bmw']
Most frequent words in:  CarlZeissMeditec-AnnualRep